In [79]:
import numpy as np
import flask
import io

import json
from keras.models import model_from_json, load_model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None

In [80]:
def load_model():
    from keras.models import model_from_json
    global model
    # Model reconstruction from JSON file
    with open('D:\\Hackathon\\trained_model\\model.json', 'r') as f:
        model = model_from_json(f.read())
        
    model.load_weights('D:\\Hackathon\\trained_model\\model.h5')

In [81]:
def prepare_data(symptoms):
    import keras
    print("code related to preprocessing the get request will be put here")
    docs = ["headache","nausea","vomiting","high fever","diarrhea","fatigue","muscle aches","coughing","pain","stiffness",
           "swelling","redness","wheezing","learning disability","epilepsy","chronic pain","blood discharge","sneezing",
           "pain during urination","genital pain","palpitation","weakness","faster heartbeat", "nasal dryness","sore throat",
           "abdominal pain","weight loss","chills","chest pain","back pain","tired","blindness","blurred vision","dry mouth",
           "paralysis","dizziness","memory loss","nose bleed","jaundice","constipation"]

    max_review_length = 200
    vectorizer = CountVectorizer()
    # tokenize and build vocab
    vectorizer.fit(docs)
    print(vectorizer.vocabulary_)
    symptoms_list = []
    symptoms_list.append(symptoms)
    X_vector = vectorizer.transform(symptoms_list)
    X = X_vector.toarray()
    print(X)
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=max_review_length, dtype='object', 
                                           padding='pre', truncating='pre', value=0.)
    print("done preprocessing!!")
    return X

In [82]:
def predict_diseases(predictions):
    encoder_Y = LabelEncoder()
    diseases = ["arthritis","asthma","ASD","cancer","chlamydia","ebola","diabetes","malaria","HIV","cirrhosis","migraine", 
                "heart disease","rhinovirus","influenza","stroke","alzheimer disease","tuberculosis","thyroid"]
    encoder_Y.fit(diseases)
    encoded_docs = encoder_Y.transform(diseases)
    # convert integers to dummy variables (i.e. one hot encoded)
    Y = np_utils.to_categorical(encoded_docs)
    print(predictions)
    predicted_probability = np.amax(predictions)
    predicted_probability = float("{0:.2f}".format(predicted_probability*100))
    arg = np.argmax(predictions)
    print(arg)
    predicted_disease = encoder_Y.inverse_transform(arg)
    print(predicted_disease)
    predictions = {'probability':predicted_probability, 'disease':predicted_disease}
#     predictions['probability']= str(predicted_probability)
#     predictions['disease']= str(predicted_disease)
    return predictions

In [ ]:
@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the
    # view
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        try:
            post_data = flask.request.get_json(force=True)
            print(post_data.get("symptoms"))
            # preprocess the image and prepare it for classification
            X = prepare_data(str(post_data.get("symptoms")))
            
            data["predictions"] = []
            # classify the input image and then initialize the list
            # of predictions to return to the client
            predicted = model.predict(X)
            predictions = predict_diseases(predicted)
            r = {"probability": str(predictions['probability']), "disease": predictions['disease'] }
            data["predictions"].append(r)
            print("Prediction done")
            
#             # loop over the results and add them to the list of
#             # returned predictions
#             for (imagenetID, label, prob) in results[0]:
#                 r = {"label": label, "probability": float(prob)}
#                 data["predictions"].append(r)

#             # indicate that the request was a success
            data["success"] = True
        except ValueError:
            print(ValueError)

    # return the data dictionary as a JSON response
    return flask.jsonify(data)

In [ ]:
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    load_model()
    app.run(host ='0.0.0.0',port=5000)

* Loading Keras model and Flask starting server...please wait until server has fully started


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
C:\Users\SiSharma\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
127.0.0.1 - - [20/Mar/2018 19:20:12] "POST /predict HTTP/1.1" 200 -


memory loss,blurry vision
code related to preprocessing the get request will be put here
{'headache': 24, 'nausea': 34, 'vomiting': 48, 'high': 26, 'fever': 22, 'diarrhea': 12, 'fatigue': 21, 'muscle': 32, 'aches': 1, 'coughing': 11, 'pain': 36, 'stiffness': 42, 'swelling': 43, 'redness': 39, 'wheezing': 51, 'learning': 28, 'disability': 13, 'epilepsy': 19, 'chronic': 9, 'blood': 5, 'discharge': 14, 'sneezing': 40, 'during': 18, 'urination': 46, 'genital': 23, 'palpitation': 37, 'weakness': 49, 'faster': 20, 'heartbeat': 25, 'nasal': 33, 'dryness': 17, 'sore': 41, 'throat': 44, 'abdominal': 0, 'weight': 50, 'loss': 29, 'chills': 8, 'chest': 7, 'back': 2, 'tired': 45, 'blindness': 4, 'blurred': 6, 'vision': 47, 'dry': 16, 'mouth': 31, 'paralysis': 38, 'dizziness': 15, 'memory': 30, 'nose': 35, 'bleed': 3, 'jaundice': 27, 'constipation': 10}
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]
done preprocessing!!
[[9.1278198e-05 3

C:\Users\SiSharma\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
127.0.0.1 - - [20/Mar/2018 19:35:56] "POST /predict HTTP/1.1" 200 -


swelling,blood discharge,chronic pain
code related to preprocessing the get request will be put here
{'headache': 24, 'nausea': 34, 'vomiting': 48, 'high': 26, 'fever': 22, 'diarrhea': 12, 'fatigue': 21, 'muscle': 32, 'aches': 1, 'coughing': 11, 'pain': 36, 'stiffness': 42, 'swelling': 43, 'redness': 39, 'wheezing': 51, 'learning': 28, 'disability': 13, 'epilepsy': 19, 'chronic': 9, 'blood': 5, 'discharge': 14, 'sneezing': 40, 'during': 18, 'urination': 46, 'genital': 23, 'palpitation': 37, 'weakness': 49, 'faster': 20, 'heartbeat': 25, 'nasal': 33, 'dryness': 17, 'sore': 41, 'throat': 44, 'abdominal': 0, 'weight': 50, 'loss': 29, 'chills': 8, 'chest': 7, 'back': 2, 'tired': 45, 'blindness': 4, 'blurred': 6, 'vision': 47, 'dry': 16, 'mouth': 31, 'paralysis': 38, 'dizziness': 15, 'memory': 30, 'nose': 35, 'bleed': 3, 'jaundice': 27, 'constipation': 10}
[[0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]
done preprocessing!!
[[8.2

...

headache
code related to preprocessing the get request will be put here
{'headache': 24, 'nausea': 34, 'vomiting': 48, 'high': 26, 'fever': 22, 'diarrhea': 12, 'fatigue': 21, 'muscle': 32, 'aches': 1, 'coughing': 11, 'pain': 36, 'stiffness': 42, 'swelling': 43, 'redness': 39, 'wheezing': 51, 'learning': 28, 'disability': 13, 'epilepsy': 19, 'chronic': 9, 'blood': 5, 'discharge': 14, 'sneezing': 40, 'during': 18, 'urination': 46, 'genital': 23, 'palpitation': 37, 'weakness': 49, 'faster': 20, 'heartbeat': 25, 'nasal': 33, 'dryness': 17, 'sore': 41, 'throat': 44, 'abdominal': 0, 'weight': 50, 'loss': 29, 'chills': 8, 'chest': 7, 'back': 2, 'tired': 45, 'blindness': 4, 'blurred': 6, 'vision': 47, 'dry': 16, 'mouth': 31, 'paralysis': 38, 'dizziness': 15, 'memory': 30, 'nose': 35, 'bleed': 3, 'jaundice': 27, 'constipation': 10}
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
done preprocessing!!
[[2.07411855e-01 8.74950929e-05 3.

C:\Users\SiSharma\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
172.26.41.251 - - [20/Mar/2018 20:15:35] "POST /predict HTTP/1.1" 200 -


vomiting,high fever,headache
code related to preprocessing the get request will be put here
{'headache': 24, 'nausea': 34, 'vomiting': 48, 'high': 26, 'fever': 22, 'diarrhea': 12, 'fatigue': 21, 'muscle': 32, 'aches': 1, 'coughing': 11, 'pain': 36, 'stiffness': 42, 'swelling': 43, 'redness': 39, 'wheezing': 51, 'learning': 28, 'disability': 13, 'epilepsy': 19, 'chronic': 9, 'blood': 5, 'discharge': 14, 'sneezing': 40, 'during': 18, 'urination': 46, 'genital': 23, 'palpitation': 37, 'weakness': 49, 'faster': 20, 'heartbeat': 25, 'nasal': 33, 'dryness': 17, 'sore': 41, 'throat': 44, 'abdominal': 0, 'weight': 50, 'loss': 29, 'chills': 8, 'chest': 7, 'back': 2, 'tired': 45, 'blindness': 4, 'blurred': 6, 'vision': 47, 'dry': 16, 'mouth': 31, 'paralysis': 38, 'dizziness': 15, 'memory': 30, 'nose': 35, 'bleed': 3, 'jaundice': 27, 'constipation': 10}
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
done preprocessing!!
[[2.0047110e-0

172.26.44.241 - - [20/Mar/2018 20:21:37] "GET / HTTP/1.1" 404 -
C:\Users\SiSharma\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
172.26.41.251 - - [20/Mar/2018 20:58:35] "POST /predict HTTP/1.1" 200 -


chitya
code related to preprocessing the get request will be put here
{'headache': 24, 'nausea': 34, 'vomiting': 48, 'high': 26, 'fever': 22, 'diarrhea': 12, 'fatigue': 21, 'muscle': 32, 'aches': 1, 'coughing': 11, 'pain': 36, 'stiffness': 42, 'swelling': 43, 'redness': 39, 'wheezing': 51, 'learning': 28, 'disability': 13, 'epilepsy': 19, 'chronic': 9, 'blood': 5, 'discharge': 14, 'sneezing': 40, 'during': 18, 'urination': 46, 'genital': 23, 'palpitation': 37, 'weakness': 49, 'faster': 20, 'heartbeat': 25, 'nasal': 33, 'dryness': 17, 'sore': 41, 'throat': 44, 'abdominal': 0, 'weight': 50, 'loss': 29, 'chills': 8, 'chest': 7, 'back': 2, 'tired': 45, 'blindness': 4, 'blurred': 6, 'vision': 47, 'dry': 16, 'mouth': 31, 'paralysis': 38, 'dizziness': 15, 'memory': 30, 'nose': 35, 'bleed': 3, 'jaundice': 27, 'constipation': 10}
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
done preprocessing!!
[[0.1448541  0.01186425 0.11396456 0